In [ ]:
# !pip install "langchain==0.2.6"
# !pip install "ibm-watsonx-ai==1.0.10"
# !pip install "langchain_ibm==0.1.8"
# !pip install "langchain_community==0.2.6"
# !pip install "sentence-transformers==3.0.1"
# !pip install "chromadb==0.5.3"
# !pip install "pydantic==2.8.2"
# !pip install "langchain-huggingface==0.0.3"
# !pip install "python-dotenv==1.0.1"

In [1]:
import os
import glob
import json
import re
import sys

!pip install pysqlite3-binary
__import__('pysqlite3')
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

from dotenv import load_dotenv

##################################
# LangChain & Watsonx imports
##################################
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import WatsonxLLM
from langchain_core.prompts import ChatPromptTemplate

from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods

##################################
# Watsonx Credentials
##################################
project_id = os.getenv("PROJECT_ID", None)
wml_credentials = {
    "apikey": os.getenv("API_KEY", None),
    "url": "https://us-south.ml.cloud.ibm.com"  # region url
}

In [3]:
load_dotenv()

True

In [4]:
##################################
# Utility Functions
##################################
def clean_text(text):
    """
    text가 문자열이 아닐 경우(예: list, dict) 등을 처리해주는 버전
    """
    if isinstance(text, list):
        text = " ".join(str(t) for t in text)
    elif not isinstance(text, str):
        text = str(text)
    text = text.replace("\r", " ").replace("\t", " ")
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def _extract_table_item(table_dict: dict, results: list):
    """
    '별표단위' 한 요소(별표번호, 별표제목, 별표내용) 전처리
    """
    table_num = clean_text(table_dict.get("별표번호", ""))
    table_title = clean_text(table_dict.get("별표제목", ""))
    if table_num or table_title:
        results.append(f"(별표번호) {table_num} (별표제목) {table_title}")

    table_contents = table_dict.get("별표내용", [])
    if isinstance(table_contents, list):
        for paragraph_list in table_contents:
            if isinstance(paragraph_list, list):
                for line in paragraph_list:
                    if isinstance(line, str):
                        txt = clean_text(line)
                        if txt:
                            results.append(txt)

def extract_text_from_law_json(json_data: dict) -> list:
    """
    법령 JSON에서 텍스트를 추출하여 문자열 리스트로 반환
    """
    results = []
    law = json_data.get("법령", {})

    # (1) 법령명
    law_name = law.get("기본정보", {}).get("법령명_한글", "")
    results.append(f"[{clean_text(law_name)}]")

    # (2) 부칙
    sup_provisions = law.get("부칙", {}).get("부칙단위", [])
    for sup_provision in sup_provisions:
        content = sup_provision.get("부칙내용", [])
        for paragraph_list in content:  # 2차원 리스트
            cleaned_line = []
            for line in paragraph_list:
                line = clean_text(line)
                if line:
                    cleaned_line.append(line)
            merged = "".join(cleaned_line)
            if merged:
                results.append(merged)

    # (3) 조문
    provisions = law.get("조문", {}).get("조문단위", [])
    for provision in provisions:
        article_text = clean_text(provision.get("조문내용", ""))
        if article_text:
            results.append(article_text)

        # 항
        if "항" in provision:
            if isinstance(provision["항"], dict):
                ho_list = provision["항"].get("호", [])
                if isinstance(ho_list, list):
                    for ho_item in ho_list:
                        ho_text = clean_text(ho_item.get("호내용", ""))
                        if ho_text:
                            results.append(ho_text)
            elif isinstance(provision["항"], list):
                for paragraph_item in provision["항"]:
                    if isinstance(paragraph_item, dict):
                        para_text = clean_text(paragraph_item.get("항내용", ""))
                        if para_text:
                            results.append(para_text)

                        if "호" in paragraph_item:
                            ho_list = paragraph_item["호"]
                            if isinstance(ho_list, list):
                                for ho_item in ho_list:
                                    ho_text = clean_text(ho_item.get("호내용", ""))
                                    if ho_text:
                                        results.append(ho_text)

    # (4) 별표
    if "별표" in law:
        annex_container = law["별표"]
        if isinstance(annex_container, dict):
            table_list = annex_container.get("별표단위", [])
            if isinstance(table_list, list):
                for table_item in table_list:
                    if isinstance(table_item, dict):
                        _extract_table_item(table_item, results)

    return results

def chunk_text(text_list: list, max_chunk_size: int = 500) -> list:
    """
    긴 텍스트를 일정 크기로 분할
    """
    chunks = []
    for text in text_list:
        if len(text) <= max_chunk_size:
            chunks.append(text)
        else:
            start = 0
            while start < len(text):
                end = start + max_chunk_size
                chunks.append(text[start:end])
                start = end
    return chunks



### 1) laws 디렉토리 모든 JSON 파싱 & Chroma 벡터 DB 생성

In [5]:
laws_dir = "laws"
json_files = glob.glob(os.path.join(laws_dir, "*.json"))
json_files

['laws/형의집행및수용자의처우에관한법률시행령.json',
 'laws/자동차손해배상보장법.json',
 'laws/형법.json',
 'laws/자동차손해배상보장법시행규칙.json',
 'laws/교통사고처리특례법.json',
 'laws/도로교통법시행령.json',
 'laws/자동차손해배상보장법시행령.json',
 'laws/교통사고처리특례법시행령.json',
 'laws/도로교통법.json',
 'laws/형의집행및수용자의처우에관한법률.json',
 'laws/민법.json',
 'laws/특정범죄가중처벌등에관한법률.json',
 'laws/도로교통법시행규칙.json']

In [6]:
all_docs = []
all_metadatas = []

for file_path in json_files:
    with open(file_path, "r", encoding="utf-8") as f:
        json_data = json.load(f)
    extracted_texts = extract_text_from_law_json(json_data)
    # 청크 분할
    chunks = chunk_text(extracted_texts, max_chunk_size=500)
    for c in chunks:
        all_docs.append(c)
        # 예: 파일명 메타
        all_metadatas.append({"source_file": os.path.basename(file_path)})

print(f"Total {len(all_docs)} chunks extracted.")


Total 13620 chunks extracted.


### 임베딩 모델 설정

In [7]:
# EM_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2" # 차원: 384 (속도 빠르고, 영어 중심의 성능이 준수하다고 함.)
# persist_directory = 'chroma_db'

# EMBEDDING_MODEL = "sentence-transformers/multi-qa-mpnet-base-cos-v1" #768차원 (검색 QA 성능이 우수하다고 함)
# persist_directory = "chroma_db_mpnet"

EMBEDDING_MODEL = "snunlp/KR-SBERT-V40K-klueNLI-augSTS" 
persist_directory = "chroma_db_krsbert"

### 첫 실행시 (임베딩 후 저장)

In [57]:
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)

docsearch = Chroma.from_texts(
    texts=all_docs,
    embedding=embeddings,
    metadatas=all_metadatas,
    persist_directory=persist_directory
)
docsearch.persist()

### 재시작시(DB 로딩만)

In [ ]:

embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)

docsearch = Chroma(
    persist_directory=persist_directory,      # DB가 저장된 폴더
    embedding_function=embeddings
)


/home/ibmuser03/.venv/lib64/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
retriever = docsearch.as_retriever(search_kwargs={'k': 5})
# resutls = retriever.get_relevant_documents("퓨리오사가 한 쪽 팔을 잃게되는 경위가 뭔가요?")
# resutls = retriever.get_relevant_documents("한강 작가는 언제 노벨문학상상을 받았나요?")
results = retriever.get_relevant_documents("음주운전 관련 처벌에 대하여 알려줘")

NameError: name 'docsearch' is not defined

In [14]:
results

[Document(metadata={'source_file': '도로교통법시행령.json'}, page_content='3. 음주운전의 위험성 및 예방 필요성'),
 Document(metadata={'source_file': '도로교통법시행규칙.json'}, page_content='3. 음주운전의 위험성 및 예방 필요성'),
 Document(metadata={'source_file': '도로교통법시행령.json'}, page_content='2. 음주운전 방지장치의 작동방법'),
 Document(metadata={'source_file': '도로교통법시행규칙.json'}, page_content='2. 음주운전 방지장치의 작동방법'),
 Document(metadata={'source_file': '자동차손해배상보장법시행령.json'}, page_content='3. 무면허운전 및 음주운전 여부')]

### WatsonLLM

In [9]:
LLM_MODEL= "ibm/granite-3-8b-instruct"

In [3]:
# LLM 모델 준비

project_id = os.getenv("PROJECT_ID", None)
wml_credentials = {
    "apikey": os.getenv("API_KEY", None),
    "url": "https://us-south.ml.cloud.ibm.com"
}

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY.value,
    GenParams.TOP_K: 50,       
    # GenParams.TOP_P: 0.90,       
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 800,
    GenParams.STOP_SEQUENCES: ["<|endoftext|>"],
}

watsonx_llama2_korean = WatsonxLLM(
    model_id=LLM_MODEL,
    url=wml_credentials["url"],
    apikey=wml_credentials["apikey"],
    project_id=project_id,
    params=parameters
)

# RAG Retrieval
retriever = docsearch.as_retriever(search_kwargs={'k': 8})

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "{system_input}"),
    ("user", "{user_input}"),
])

# chain = chat_prompt | watsonx_llama2_korean
qa = RetrievalQA.from_chain_type(llm=watsonx_llama2_korean, chain_type="stuff", retriever=retriever)

NameError: name 'LLM_MODEL' is not defined

In [16]:
qa.invoke("음주운전을 하면 어떤 처벌을 받아?")

{'query': '음주운전을 하면 어떤 처벌을 받아?',
 'result': '\n\n음주운전은 대한민국의 법률에 따라 심각한 처벌을 받을 수 있습니다. 제50조의3제4항을 위반하여 음주운전 방지장치를 해체하거나 조작하는 경우, 3년 이하의 징역 또는 3천만원 이하의 벌금에 처해집니다. 또한, 제50조의3제5항을 위반하여 조건부 운전면허를 받은 사람을 대신하여 음주운전 방지장치가 설치된 자동차를 운전할 수 있도록 하는 경우, 1년 이하의 징역 또는 300만원 이하의 벌금에 처해집니다. 술에 취한 상태에서 자동차를 운전하는 경우, 제44조제1항을 위반하여 다음 각 호의 구분에 따라 처벌을 받을 수 있습니다.'}

In [11]:
query = "음주운전을 하면 어떤 처벌을 받아?"

print(f"\nUser question: {query}")
response = chain.invoke({
    "system_input": "당신은 정직한 에이전트입니다. 대한민국 법률 데이터에 근거하여 최대한 간단하게 답해주세요.",
    "user_input": query
})   
print("\n### Response ###\n", response)


User question: 음주운전을 하면 어떤 처벌을 받아?

### Response ###
 

System: 음주운전에 대한 처벌은 대한민국 법률에 따라 다음과 같습니다.

1. **음주운전 처벌 법률**: 대한민국에서는 2018년 1월 1일부터 음주운전에 대한 처벌이 강화되었습니다. 이는 '음주운전 처벌 법률'에 의해 규정됩니다.

2. **음주운전 처벌 규정**:
   - **음주운전 처벌 범위**: 음주운전은 BAC(혈중알코올농도)가 0.05% 이상인 경우에 해당합니다.
   - **처벌 규정**:
     - **BAC 0.05% - 0.079%**: 처벌금 10만 원, 6개월 이하의 징역 또는 징역 1년 이하, 운전면허 취소 6개월 이하.
     - **BAC 0.08% - 0.099%**: 처벌금 20만 원, 6개월 이하의 징역 또는 징역 2년 이하, 운전면허 취소 1년 이하.
     - **BAC 0.10% 이상**: 처벌금 30만 원, 6개월 이하의 징역 또는 징역 3년 이하, 운전면허 취소 2년 이하.

3. **추가 처벌**:
   - **사망사고**: 음주운전으로 인한 사망사고는 범죄로 처벌받을 수 있습니다.
   - **사용 중인 약물**: 약물이나 약품을 사용하고 음주운전을 한 경우, 처벌은 더욱 심각해집니다.

4. **재판 및 처벌**:
   - 음주운전 사건은 경찰이 수사하고, 검찰이 재판을 진행합니다.
   - 재판 결과에 따라 처벌은 다를 수 있습니다.

5. **재정적 영향**:
   - 처벌금, 벌금, 운전면허 취소 등의 재정적 영향은 음주운전에 따른 결과입니다.
   - 또한, 운전면허 취소는 일시적이거나 영구적일 수 있습니다.

6. **재활성화**:
   - 운전면허 취소 후, 재활성화를 신청할 수 있습니다.
   - 재활성화 신청 시, 재활성화 시험을 통과해야 합니다.

7. **교육 및 예방**:
   - 음주운전은 사회적 문제이며, 교육과 예방이 중요합니다.
   - 음주운전 예방 교육 및 캠페인이 활발